# Data Gathering for Projections 

## Setup 

In [56]:
#import libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json

## Functions to Pull Data

### Function to Pull Data from a Week 

In [102]:
def pull_week(year, week):
    #set scrape url
    base = 'https://www.fantasyfootballdatapros.com/api/players/'
    pull_url = f'{base}{str(year)}/{str(week)}'
    
    #scrape from the site
    print(f"Scraping Week {week} of {year}...")
    raw = requests.get(pull_url)
    
    #extract json data
    soup = BeautifulSoup(raw.text, 'lxml')
    text = soup.find('p').text
    raw_player_info = json.loads(text)
    
    return raw_player_info    

### Function to build player dataframe from json data

In [108]:
def build_player_df(raw_data):
    clean_player_info = []
    
    for player in raw_data:
        new_player = {}
    
        #directly transferred cols
        new_player['name'] = player['player_name']
        new_player['position'] = player['position']
        new_player['team'] = player['team']
        new_player['fumbles'] = player['fumbles_lost']

        #extract stats info
        new_player['ints'] = player['stats']['passing']['int']
        new_player['pass_td'] = player['stats']['passing']['passing_td']
        new_player['pass_yds'] = player['stats']['passing']['passing_yds']
        new_player['rec_tds'] = player['stats']['receiving']['receiving_td']
        new_player['rec_yds'] = player['stats']['receiving']['receiving_yds']
        new_player['carries'] = player['stats']['rushing']['rushing_att']
        new_player['rush_td'] = player['stats']['rushing']['rushing_td']
        new_player['rush_yds'] = player['stats']['rushing']['rushing_yds']

        #extract fantasy points
        new_player['pts_std'] = player['fantasy_points']['standard']
        new_player['pts_half'] = player['fantasy_points']['half_ppr']
        new_player['pts_ppr'] = player['fantasy_points']['ppr']

        clean_player_info.append(new_player)
        
    return pd.DataFrame(clean_player_info)

### Cleaning Functions

In [112]:
#deal with weird position cases
def fix_odd_pos(x):
    if x in ['WR', 'RB', 'TE', 'QB']:
        return x
    elif x in ['FB', 'WR/RS', 'HB', 'FB/DL', ]:
        return 'RB'
    elif x in ['WR/PR', 'CB', 'DB', 'DB/LB']:
        return 'WR'
    elif x in ['C']:
        return 'TE'
    else:
        return "ERROR CHECK ME"

In [113]:
#apply other cleaning functions
def clean_player_df(df):
    df['position'] = df['position'].apply(lambda x: fix_odd_pos(x))
    return df

### Export Functions

In [129]:
def export_df(df,year, week):
    try:
        filename = f'./data/{year}-wk{week}-players.csv'    #dynamic file naming
        df.to_csv(filename, index=False)               #export dataframe as csv
        return df
    except:
        print("Error occurred.")                       #except statement in case of major error
        return df

### All together function

In [132]:
#function to run all others
def weekly_data_pull(year, week):
    raw = pull_week(year, week)
    df = build_player_df(raw)
    df = clean_player_df(df)
    return export_df(df, year, week)

## Iterate through weeks to pull data

In [134]:
#iterate through all 13 weeks of the 2019 season
for week_no in range(1,14):
    weekly_data_pull(year, week_no)

Scraping Week 1 of 2019...
Scraping Week 2 of 2019...
Scraping Week 3 of 2019...
Scraping Week 4 of 2019...
Scraping Week 5 of 2019...
Scraping Week 6 of 2019...
Scraping Week 7 of 2019...
Scraping Week 8 of 2019...
Scraping Week 9 of 2019...
Scraping Week 10 of 2019...
Scraping Week 11 of 2019...
Scraping Week 12 of 2019...
Scraping Week 13 of 2019...
